# Описательная статистика in R. Пример 2
## Категориальный тип данных
**Задача.** В исследованиях применимости комплекса витаминов и минералов "Мультитабс" для профилактики гриппа в основной группе приняло участие 160 человек, а в контрольной - 100 человек. Во время сезонной эпидемии гриппа было выявлено 5 случаев заражений в основной группе и 9 в контрольной. Вычислите для обеих выборок долю и доверительные интервалы.

### 1.	Определение типа переменных
Т.к. спектр значений обеих переменных конечен (2 категории: здоров, болен), то это качественный тип данных

### 2.	Ввод данных в R
Подготовьте данные в CSV (comma-separated value) формате для создания матрицы. Используйте функцию `read.csv`

In [1]:
Input = ("
         Случаи, Основная группа, Контрольная группа
         Больных, 5, 9
         Здоровых, 155, 91"
        )
Data = as.data.frame(read.csv(textConnection(Input), header = TRUE, row.names = 1))
Data

,Основная.группа,Контрольная.группа
Больных,5,9
Здоровых,155,91


### 2.	Вычисление долей
Для получения долей для групп в переменной `Data`, необходимо преобразовать `Data` в матрицу и использовать её в качестве аргумента функции `prop.table` с указанием дополнительного аргумента `margin = 2`. Установка аргумента `margin` в значение 2 указывает функции, что необходимо вычислить доли по отношению к сумме колонок в исходной матрице (количество случаев в каждой группе).

In [2]:
Data.prop = prop.table(as.matrix(Data), margin = 2)
Data.prop

,Основная.группа,Контрольная.группа
Больных,0.03125,0.09
Здоровых,0.96875,0.91


### 3.	Вычисление доверительных интервалов долей
Доверительный интервал может быть вычислен с помощью функции `binom.test`, применённой для одной выборки

In [3]:
ог_итог = binom.test(Data$Основная.группа[1], sum(Data$Основная.группа))
ог_итог


	Exact binomial test

data:  Data$Основная.группа[1] and sum(Data$Основная.группа)
number of successes = 5, number of trials = 160, p-value < 2.2e-16
alternative hypothesis: true probability of success is not equal to 0.5
95 percent confidence interval:
 0.01022314 0.07141770
sample estimates:
probability of success 
               0.03125 


In [4]:
кг_итог = binom.test(Data$Контрольная.группа[1], sum(Data$Контрольная.группа))
кг_итог


	Exact binomial test

data:  Data$Контрольная.группа[1] and sum(Data$Контрольная.группа)
number of successes = 9, number of trials = 100, p-value < 2.2e-16
alternative hypothesis: true probability of success is not equal to 0.5
95 percent confidence interval:
 0.0419836 0.1639823
sample estimates:
probability of success 
                  0.09 


Из результатов выше видно, что *95% доверительный интервал доли* для Основной группы составил диапазон [0,01; 0,07], а для Контрольной группы *95% доверительный интервал доли* составил диапазон значений [0,04; 0,16]

### 4.	Подготовка результатов
Давайте соберём результаты из предыдущих пунктов в одном месте. Для этого мы создадим два списка со значениями `Доля`, `нжн.ДИ`, `вер.ДИ`, соответствующие значениям *доли*, *нижняя граница доверительного интервала*, *верхняя граница доверительного интервала*.

In [5]:
кг_итог = list(Доля = кг_итог$estimate,
               нижн.ДИ = кг_итог$conf.int[1],
               верх.ДИ = кг_итог$conf.int[2])
ог_итог = list(Доля = ог_итог$estimate, 
               нижн.ДИ = ог_итог$conf.int[1], 
               верх.ДИ = ог_итог$conf.int[2])

Итоговые данные конструируются объединением колонок обеих групп с помощью функции `cbind`
Final data is being constructed by column combine of both control group and main group lists via `cbind` function.

In [6]:
Data.DescrStat = as.data.frame(cbind(Контрольная.группа = кг_итог, Основная.группа = ог_итог))
Data.DescrStat

,Контрольная.группа,Основная.группа
Доля,0.09,0.03125
нижн.ДИ,0.0419836,0.01022314
верх.ДИ,0.1639823,0.0714177


### 5.	Интерпретация данных
Интерпретацию необходимо начинать со сравнения долей, после чего проверять перекрываются ли доверительные интервалы (ДИ) групп. Если ДИ перекрываются, то вывод должен быть сформулирован в виде гипотезы. В противном случае можно сделать твёрдый вывод, хотя для точности и скрупулёзности подхода рекомендуется даже в этом случае давать вывод в качестве гипотезы.

Как видно из данных выше, в *Контрольной группе* доля заболевших людей выше, чем в *Основной группе*. Это свидетельствует в пользу утверждения о благоприятном эффекте от приёма рассматриваемого препарата для профилактики гриппа.
Доверительные интервалы групп пересекаются (верхняя граница ДИ Основной группы находится внутри ДИ Контрольной группы), а значит мы не можем твёрдо утверждать о наличии профилактического эффекта препарата, но мы можем сформулировать это утверждение в форме медицинской гипотезы.

### Вывод:
Приём препарата может предотвратить инфицирование гриппом почти в 3 раза по сравнению с плацебо.

**Примечание:** слово "может" в выводе это лишь один из многих способов формулирования вывода в виде гипотезы